##### Notebook for analysis of a PCA-ANN based combustion model in diesel engine conditions  
##### Ref: D.K. Dalakoti, A. Wehrfritz, B. Savard, M.S. Day, J.B. Bell, E.R. Hawkes, An a priori evaluation of a principal component and artificial neural network based combustion model in diesel engine conditions, Proc. Combust. Inst., https://doi.org/10.1016/j.proci.2020.06.263, (2020). 

In [ ]:
import numpy as np
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import axes3d, Axes3D 
import cantera as ct
from cantera import ck2cti
import os
import time
from helper_functions import read_data, read_reaction, do_normalization, do_inverse_norm, get_table_noholes, opt_est
from neural_networks import get_model_species, get_model_reac, get_model_prop
from helper_functions import get_cp, get_viscosity, get_conductivity, get_diffusion
from neural_networks import training

In [ ]:
#Read the data for
# 2D Data
data = read_data('./RR_data/2D/data2d_lower2.bin',56)
reac = read_reaction('./RR_data/2D/reac2d_lower2.bin')

# Additional 2D for a different scalar dissipation rate case
#data2 = read_data('./RR_data/data2d_base.bin',56)
#reac2 = read_reaction('./RR_data/reac2d_base.bin')

# Training can also be done using data from 0D reactors
#data = np.fromfile('../PCA/data_0D.bin',dtype=np.single)
#data = np.reshape(data,(56,int(data.size/56)))   
#data = data.T
#data = np.delete(data,0,1)

#reac = np.fromfile('../PCA/reac_0D.bin',dtype=np.single)
#reac = np.reshape(reac0D,(56,int(reac.size/56)))   
#reac = reac.T
#HRR = reac[:,0]
#reac = np.delete(reac,0,1)
#reac[:,53]=HRR
#reac = np.delete(reac,54,1)

# Training can also be done using 1D nonpremixed flamelets
#data = get_data_2d('../PCA/data_25.h5')
#reac = get_reac_2d('../PCA/data_25.h5')


# 3D Data
dat3d = read_data('./RR_data/3D/dat3d.bin',56)
reac3d = read_reaction('./RR_data/3D/reac3d.bin')


In [ ]:
# Normalize data
nc=54
datanorm = do_normalization(data[:,0:nc],data[:,0:nc],'range')
dat3dnorm = do_normalization(dat3d[:,0:nc],data[:,0:nc],'range')

In [ ]:
# Do PCA
nc=54
pca = PCA(n_components=5)
Xt= pca.fit_transform(datanorm[:,0:nc])
components = pca.components_
# Subtract training mean because we are using training data to find PCA components
Xt3d = np.matmul((dat3dnorm[:,0:nc]-np.mean(datanorm[:,0:nc],0)),components.T)
np.sum(pca.explained_variance_ratio_)
#XPCA = pca.inverse_transform(Xt)


In [ ]:
# Find the PC reaction rates
RPCA2D = np.matmul(reac[:,0:nc]/(np.max(data[:,0 :nc],0)-np.min(data[:,0:nc],0)),components.T)
RPCA3D = np.matmul(reac3d[:,0:nc]/(np.max(data[:,0:nc],0)-np.min(data[:,0:nc],0)),components.T)

In [ ]:
# Train neural network to learn species mass fractions
model = get_model_species(5,53)
trainSpec = training(model, Xt[:,0:5], data[:,0:53], 'std')
trainSpec.do_training(2056,100)
e1, e2 = trainSpec.get_errors(Xt3d, dat3d[:,0:53])
sorted(list(zip(specs,e2)), key=lambda a: a[1],reverse=True)

In [ ]:
# Train neural networks for prediction of PC reaction rates, one model for each RR
trainRR = []
errR2 = []
errMSE = []
pred = np.zeros((dat3d.shape[0],5))
for i in range(0,5):
    model = get_model_reac(5)
    trainRR.append(training(model, Xt[:,0:5], RPCA2D[:,i:i+1], 'std'))
    trainRR[i].do_training(1024,100)
    #e1, e2 = trainRR[i].get_errors(Xt3d, RPCA3D[:,i:i+1])
    #errR2.append(e1)
    #errMSE.append(e2)
    pred[:,i:i+1] = trainRR[i].get_predictions(Xt3d)
    
    

In [ ]:
# Get cp for 2d and 3d dataset
cp2d = get_cp(data)
cp3d = get_cp(dat3d)

# Train neural network to cp
model = get_model_prop(5,1)
traincp = training(model, Xt[:,0:5], cp2d[:,None], 'std')
traincp.do_training(1024,100)
e1, e2 = traincp.get_errors(Xt3d, cp3d[:,None])

# Similarly get lambda, mu and mean molecular weights



In [ ]:
# get diffusion coefficents of species
D2D = get_diffusion(data[:,0:54])
D3D = get_diffusion(dat3d[:,0:54])
D2D = np.column_stack((D2D,cp2d))
D3D = np.column_stack((D3D,cp3d))
# Convert to diffusion coefficients of PCs
DDPCA2D = np.matmul(D2D,components.T)
DDPCA3D = np.matmul(D3D,components.T)


model = get_model_prop(5,5)
trainDiff = training(model, Xt[:,0:5], DDPCA2D, 'std')
trainDiff.do_training(1024,100)
e1, e2 = trainDiff.get_errors(Xt3d, DDPCA3D)



In [ ]:
# optimal estimators for PC reaction rates
opt_est_RR = np.zeros((RPCA3D.shape[0],5))
for i in range(0,1):

    opt_est_RR[:,i] = opt_est(Xt3d,RPCA3D[:,i],100)
    

In [ ]:
# Get predictions of HRR based on conventional tabulation with mixture fraction and progress var
HRR2d = np.copy(reac[:,53])
HRR3d = np.copy(reac3d[:,53])
idx_Y = [27,28,30,31] # CO + CO2 + H2 + H2O
Yc2d = np.sum(data[:,idx_Y],axis=1)
Yc3d = np.sum(dat3d[:,idx_Y],axis=1)
xi2d = np.copy(data[:,54]) # Mixture fraction
xi3d = np.copy(dat3d[:,54])
Xtab2d = np.append(Yc2d[:,None],xi2d[:,None],axis=1)
Xtab3d = np.append(Yc3d[:,None],xi3d[:,None],axis=1)

pred = get_table_noholes(Xtab2d, HRR2d[:,None], Xtab3d, 200)

# prediction based on first two PC based on conventional tabulation method
predPCA = get_table_noholes(Xt[:,0:2], HRR2d[:,None], Xt3d[:,0:2], 200)




In [ ]:
# Code for plotting Fig. 4 in paper. To be used once neural networks are trained
# Assuming pred contains neural network predictions
# Sample code for OH mass fraction

plt.figure(figsize=(3,3))
plt.scatter(dat3d[0:-1:100,42]/np.max(dat3d[0:-1:100,42]), pred[0:-1:100,42]/np.max(dat3d[0:-1:100,42]),marker='.',c='k',s=10)
plt.locator_params(axis='y', nbins=3)
plt.locator_params(axis='x', nbins=3)
x = np.linspace(0.0,1.0,10)
plt.plot(x,x,'r--')
plt.xlabel('$DNS$')
plt.ylabel('$ANN$')
plt.title('$Y(OH)$')
plt.xlim((0.0,1.0))
plt.ylim((0.0,1.0))
plt.grid(alpha=0.2)
ax=plt.gca()
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(25)

#plt.savefig('Final/OH2.png',dpi=300,bbox_inches = "tight")

In [ ]:
from helper_functions import cond_mean
# Error in the prediction of reaction rates, for Figure 3
maxs = np.max(np.abs(RPCA3D),axis=0)
eps = np.sum((RPCA3D-pred)**2,axis=1)
nbins=100
condAvg = cond_mean(Xt3d[:,0:2],eps,nbins)
#condAvg = np.sqrt(condAvg)
RO2 = cond_mean(Xt3d[:,0:2],dat3d[:,3],nbins)
RO2[np.isnan(RO2)] = 0

In [ ]:
#Code to plot Figure 3
plt.figure()
x = np.linspace(0,1,nbins)
y = np.linspace(0,1,nbins)
plt.contourf(x,y,np.sqrt(condAvg)/np.sum(maxs),cmap='jet',levels=100)
plt.colorbar()
plt.contour(x,y,RO2, levels=[0.0002],colors='red')

#plt.clim(0,0.15)
#plt.colorbar()
